In [6]:
#%% ----------------------------------- Package Imports
import pandas as pd
from numpy import nan
import dash
import dash_table
from fredapi import Fred
fred = Fred(api_key='287dcf1a4ec41e9f44645fb74180eb42')
import pandas as pd
import numpy as num
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox, Layout
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
plt.style.use('seaborn')
%config InlineBackend.figure_format = 'svg'
from __future__ import print_function
from bqplot import topo_load
from bqplot.interacts import panzoom
import datetime as dt

In [ ]:
def pv(cash_flow, period, day_count, cost_of_capital):
    #day_count is meant to proxy the length of the "period" the cash flow is received in 
    v = cash_flow/((1+cost_of_capital)**(period*day_count))
    return v

In [8]:
from ipywidgets import interactive,interact, HBox, Layout,VBox
#function to plot the different curves
def plot_function(u=1,v=2,w=3,x=4,y=5,z=6):
    time=np.arange(0,1,0.01)
    df=pd.DataFrame({"Y1":np.sin(time*u*2*np.pi),"y2":np.sin(time*v*2*np.pi),"y3":np.sin(time*w*2*np.pi),
                    "y4":np.sin(time*x*2*np.pi),"y5":np.sin(time*y*2*np.pi),"y6":np.sin(time*z*2*np.pi)})
    df.plot()
widget=interactive(plot_function,u=1,v=2,w=3,x=4,y=5,z=6)
controls = HBox(widget.children[:-1], layout = Layout(flex_flow='row wrap'))
output = widget.children[-1]
display(VBox([controls, output]))

In [4]:
df = fred.search('US rates')
USrates = df.loc[df['frequency']=='Monthly']
#df['title'].T
#df = df['title'].T
#df.columns
#df.loc(df['frequency_short']=='M')
#df = df.loc[df['frequency']=='Monthly'].T

In [11]:
x = widgets.Select(options=USrates['title'].tolist(), layout=Layout(width= '600px'))
y = widgets.IntSlider(
    value=3,
    min=0,
    step=1,
    description='RSI Window:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d') 
#display(x)

def g(x,y):
    
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.collections import LineCollection
    from matplotlib.colors import ListedColormap, BoundaryNorm
    
    plot_series = USrates.loc[USrates['title']==x].index[0]
    #print(plot_series)
    def generate_RSI_stats(df, series,RSI_decay=3):
        df = pd.DataFrame(data = fred.get_series(series),columns =[series])
        dff = 'diff_'+series
        dn = series+'_dn'
        up = series+'_up'
        df[dff] = df[series].diff()
        df[up] = df[dff]
        df.loc[df[dff] < 0, up] = 0
        df[up] = (df[up]).ewm(alpha=(1/RSI_decay)).mean()
        df[dn] = df[dff]
        df.loc[df[dff] > 0, dn] = 0
        df[dn] = (df[dn]).ewm(alpha=(1/RSI_decay)).mean()
        df[series+'_RSI_signal'] = 100-100/(1-df[up]/df[dn])
        df = df[[series,series+'_RSI_signal']]
        return df
    
    plots = generate_RSI_stats(df = fred.get_series(plot_series), series=plot_series,RSI_decay=y).plot(subplots=True)
    plt.show()
        
interact(g, x=x, y=y);


AttributeError: 'Select' object has no attribute 'Layout'

In [7]:
@interact(cash_flow=widgets.FloatText(
    value=100,
    description='Initial Cash Flow:',
    disabled=False
),period=widgets.FloatText(
    value=1,
    description='Discount Periods:',
    disabled=False
), day_count=widgets.FloatText(
    value=1,
    description='Day Count:',
    disabled=False
), cost_of_capital=widgets.FloatText(
    value=.07,
    description='Cost of Capital:',
    disabled=False
))
def pv(cash_flow, period, day_count, cost_of_capital):
    #day_count is meant to proxy the length of the "period" the cash flow is received in 
    v = cash_flow/((1+cost_of_capital)**(period*day_count))
    return v

interactive(children=(FloatText(value=100.0, description='Initial Cash Flow:'), FloatText(value=1.0, descripti…